# Reproduce results from Matsuoka et al. (2009)

This article focuses on oblique wave propagation through anisotropic ice. The results are plotted as a power differential (i.e., birefringent power losses) but the model can be used to calculate phase in the same way it has been used for non-oblique waves.

Reference:

Matsuoka, K., Wilen, L., Hurley, S. P., & Raymond, C. F. (2009). Effects of birefringence within ice sheets on obliquely propagating radio waves. IEEE Transactions on Geoscience and Remote Sensing, 47 (5), 1429–1443. doi: 10.1109/TGRS.2008.2005201

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB

%matplotlib inline

In [ ]:
### Some constants are fixed for all model realizations in this particular figure

# geometry in the vertical
d = 1e3
dz = 1e3

# azimuthal rotation of the material anisotropy
ψ = [90.*np.pi/180.]
# range of orientations of the instrument based on antenna offset and rotation
θw,ψw = np.meshgrid(np.linspace(0,40,200)*np.pi/180.,np.linspace(0,90,200)*np.pi/180.)

# get the amplitude of the returned signal as in Matsuoka et al. (2009)
def matsuoka_amp_return(em,idctx,θ,χ,fc,
                        θw=θw,ψw=ψw,
                        d=d,dz=dz,ψ=ψ):
    # empty array to fill
    AMP = np.empty_like(θw)
    for i in range(len(θw)):
        for j in range(len(θw[0])):
            # assign instrument properties to the model
            em.system_setup(fc=fc, psi_w=ψw[i,j], theta_w=θw[i,j])
            # assign material properties to the model
            em.ice_properties(idctx=idctx,theta=θ[0],psi=ψ[0],chi=χ[0])
            # solve hte model
            em.single_depth_solve(d,[dz],θ,ψ,χ,gamma=[1., 1.],idctx=idctx)
            # assign the real component to the output array
            AMP[i,j] = np.real(10.*np.log10(em.S[0,0]**2.))
    return AMP

# plot a figure with separate panels as in Matsuoka et al. (2009)
def plot_matsuoka_output(idctx,fcs,θs,χs,ψ=ψ):
    
    plt.figure(figsize=(12,6))
    # eight subpanels
    for ii in range(8):
        # instantiate the model class
        em = effective_medium()
        # get the zenith orientations of the material
        θ = θs[ii]*np.pi/180.
        # calculate the returning amplitude with the prior function
        AMP = matsuoka_amp_return(em,idctx,[θ],np.array([χs[ii]]),fcs[ii])
        # plot the result
        ax = plt.subplot(2,4,ii+1)
        plt.pcolormesh(ψw*180./np.pi,θw*180./np.pi,AMP,cmap='Greys_r', vmin=-5,vmax=0)
        if ii == 4:
            plt.xlabel('Azimuth Angle (deg)')
            plt.ylabel('Polar Angle (deg)')
    plt.tight_layout()

In [ ]:
### Figure 6 top rows

# center frequencies
fcs = [1e8,5e8,1e9,2e9,1e8,5e8,1e9,2e9]
# zenith tilts
θs = [5,5,5,5,10,10,10,10]
# COF eigenvalues
χ = np.array([[0,0,1]])
χs = np.tile(χ,(8,1))

plot_matsuoka_output('uniaxial',fcs,θs,χs)

In [ ]:
### Figure 6 bottom rows

plt.figure(figsize=(12,6))

# center frequencies
fcs = [1e8,5e8,1e9,2e9,1e8,5e8,1e9,2e9]
# zenith tilts
θs = np.zeros(8)
# magnitude of anisotropy (biaxial)
dχs = [.025,.025,.025,.025,.1,.1,.1,.1]
# loop through all the anisotropy magnitudes to get eigenvalues for biaxial
χs = np.empty((8,3))
for ii in range(8):
    χs[ii] = np.array([(.3+dχs[ii])/2.,(.3-dχs[ii])/2.,0.7])

plot_matsuoka_output('biaxial',fcs,θs,χs)